<a href="https://colab.research.google.com/github/tomoko0123/c3/blob/master/partners_contest_xlsx_encryption.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## エクセルのパスワードロック
### 使い方
1. driveの指定フォルダにエクセル(.xlsx)を置きます。パスワードはフォルダ名の「~#password.xlsx」のパスワード部分になります。
1. 関数を実行します。（Ctrl + F9）
1. `INPUT_FOLDER_PATH/ファイル名/参加者データ.xlsx`の名前でパスワード付きエクセルが生成されます。

### 環境設定
- INPUT_FOLDER_PATH：元ファイルのパス。
- OUTPUT_FOLDER_PATH：パスワードロックしたエクセルファイルの出力先

### 実装について
secure-spreadsheetという、`xlsx-populate`のラッパーライブラリを使っている。

https://github.com/ankane/secure-spreadsheet/tree/master

In [9]:
# secure-spreadsheetの導入
!npm install -g secure-spreadsheet

# ヘッダ削除のためのライブラリ
!pip install openpyxl



added 22 packages, and audited 23 packages in 4s

found 0 vulnerabilities


In [10]:
# Googleドライブのマウント
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [13]:
# https://github.com/ankane/secure-spreadsheet
# 特定のディレクトリのxlsxにパスワードをかける
import os
import subprocess
import re
import openpyxl

# 出力元フォルダの指定。
# 共有フォルダのフォルダパス
BASE_FOLDER_PATH = "/content/drive/Shareddrives/提携コンクール専用ドライブ/77_提携コンクール/【共有】実施事務局用"
INPUT_FOLDER_PATH = f"{BASE_FOLDER_PATH}/参加者データ/元データ" # e.g.  "/content/drive/MyDrive/input"

# ================関数定義================
# ヘッダ行を削除（FileMakerから書き出す時に重複しているため。）
def delete_header_row(input_file_path):
    wb = openpyxl.load_workbook(input_file_path)
    ws = wb.active
    ws.delete_rows(1)
    wb.save(input_file_path)

def delete_file(file_path):
    try:
        os.remove(file_path)
        return f"File '{file_path}' deleted successfully!"
    except FileNotFoundError:
        return f"Error: File '{file_path}' not found!"
    except PermissionError:
        return f"Error: Permission denied to delete '{file_path}'!"
    except Exception as e:
        return f"Error: {e}"

# 暗号化
def encrypt_file(input_file_path, output_file_path, password):
    cat_process = subprocess.Popen(["cat", input_file_path], stdout=subprocess.PIPE)

    secure_spreadsheet_process = subprocess.Popen(
        ["secure-spreadsheet", "--password", password, "--input-format", "xlsx"],
        stdin=cat_process.stdout,
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE
    )

    with open(output_file_path, "wb") as f:
        while True:
          # 1024byteで区切る
            data = secure_spreadsheet_process.stdout.read(1024)
            if not data:
                break
            f.write(data)

    cat_process.wait()
    secure_spreadsheet_process.wait()
    stdout, stderr = secure_spreadsheet_process.communicate()
    if stderr:
        print("Error:", stderr.decode())

def validate_filename_format(filename):
    """
    ファイル名は次の規則に従うこと '<filename>#<password>.xlsx'
    """
    return re.match(r'^(.*)#(.*)\.xlsx$', filename) is not None
# ===================================

if not os.path.isdir(INPUT_FOLDER_PATH):
    raise FileNotFoundError(f"フォルダが存在しません。INPUT_FOLDER_PATHが存在するか確認してください。： '{INPUT_FOLDER_PATH}'")

# 先にファイル規則をassertする。
for filename in os.listdir(INPUT_FOLDER_PATH):
    if filename.endswith('.xlsx') and not validate_filename_format(filename):
        raise ValueError(f"ファイル名「{filename}」が不正です。ファイル名規則： '<filename>#<password>.xlsx'")

for filename in os.listdir(INPUT_FOLDER_PATH):
    if filename.endswith('.xlsx'):
        # ファイル名のうち、「#」以降をパスワードとする
        match = re.match(r'^(.*)#(.*)\.xlsx$', filename)
        if match:
            destination = match.group(1).replace("_", "/")

            # フォルダはファイル名から取る
            # フォルダは先にFileMakerで作っておく
            OUTPUT_FOLDER_PATH = f"{BASE_FOLDER_PATH}/{destination}" # e.g.  "/content/drive/MyDrive/output"
            if not os.path.exists(OUTPUT_FOLDER_PATH):
              raise ValueError(f"先にフォルダを作成してください。フォルダ名：{OUTPUT_FOLDER_PATH}")

            input_file_path = os.path.join(INPUT_FOLDER_PATH, filename)
            output_filename = f"申込データ_{match.group(1)}.xlsx"
            output_file_path = os.path.join(OUTPUT_FOLDER_PATH, output_filename)
            password = match.group(2)

            # エクセルのヘッダを削除
            print(f"Deleteing header row from {filename}")
            delete_header_row(input_file_path)

            # 暗号化
            print(f"Encrypting {filename} with password {password}")
            encrypt_file(input_file_path, output_file_path, password)
            print(f"{filename} encrypted to {output_file_path}")

            # 元ファイルを削除する。残っていればエラーの可能性あり。
            delete_file(input_file_path)

        else:
            raise ValueError(f"File {filename} does not match the pattern '<filename>#<password>.xlsx'")


申込データ_2024_ブルグミュラーコンクール 2024 東京11月大会_1004巣鴨1（高校・大人限定）土屋テスト.xlsx
/content/drive/Shareddrives/提携コンクール専用ドライブ/77_提携コンクール/【共有】実施事務局用/2024/ブルグミュラーコンクール 2024 東京11月大会/1004巣鴨1（高校・大人限定）土屋テスト/申込データ_2024_ブルグミュラーコンクール 2024 東京11月大会_1004巣鴨1（高校・大人限定）土屋テスト.xlsx
Deleteing header row from 2024_ブルグミュラーコンクール 2024 東京11月大会_1004巣鴨1（高校・大人限定）土屋テスト#0339441581.xlsx


/usr/local/lib/python3.10/dist-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


Encrypting 2024_ブルグミュラーコンクール 2024 東京11月大会_1004巣鴨1（高校・大人限定）土屋テスト#0339441581.xlsx with password 0339441581
2024_ブルグミュラーコンクール 2024 東京11月大会_1004巣鴨1（高校・大人限定）土屋テスト#0339441581.xlsx encrypted to /content/drive/Shareddrives/提携コンクール専用ドライブ/77_提携コンクール/【共有】実施事務局用/2024/ブルグミュラーコンクール 2024 東京11月大会/1004巣鴨1（高校・大人限定）土屋テスト/申込データ_2024_ブルグミュラーコンクール 2024 東京11月大会_1004巣鴨1（高校・大人限定）土屋テスト.xlsx
